連接 drive

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/Colab_Notebooks/HW3'
##進入專案目錄
os.chdir(path) #切換該目錄
os.listdir() #確認目錄內容

下載資料

In [ ]:
os.chdir(path + '/data')
!gdown '19b-RnXuAFhXhEHW7ah0M-pQsxHr1cPsZ'
!unzip -o hw3data.zip

執行 preprocess

In [ ]:
os.chdir(path + '/src')
!python preprocess.py -d ../data/train.jsonl -t ../cache/train.json && python preprocess.py -d ../data/public.jsonl -t ../cache/dev.json && python preprocess.py -d ../data/public.jsonl -t ../cache/test.json --no-ans
!python preprocess.py -d ../data/sample_test.jsonl -t ../cache/sample_test.json --no-ans

安裝模型訓練相依套件

In [ ]:
os.chdir(path + '/libs/summarization')
!pip install -r requirements.txt
!pip install transformers

訓練模型

In [ ]:
os.chdir(path + '/src')
!bash process.sh ../cache/train.json ../cache/dev.json ../model ../model/model 5

測試模型

In [ ]:
os.chdir(path + '/src')
!python test.py -m ../model/model -t ../cache/test.json -d ../cache/submission.jsonl

實驗生成策略

In [ ]:
import json
from transformers import pipeline
os.chdir(path + '/src')
summarizer = pipeline("summarization", model='../model/model', device=0, batch_size=4,num_beams=1)
with open('../cache/test.json') as f:
  for line in f:
    obj = json.loads(line)
    print(summarizer(obj['text']))

計分

In [ ]:
# 安裝套件
os.chdir(path + '/libs/rouge')
!pip install -e tw_rouge

In [ ]:
# 執行計分腳本
os.chdir(path + '/libs/rouge')
!python eval.py -r ../../data/public.jsonl -s ../../cache/submission.jsonl